In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time
import warnings
import json
warnings.filterwarnings('ignore')

In [2]:
youtube = pd.read_csv("./data/USvideos.csv")

In [3]:
with open("./data/US_category_id.json", "r") as f:
    categories = json.load(f)

In [5]:
cat_dict = dict()
for item in categories["items"]:
    cat_dict[item["id"]] = item["snippet"]["title"]

In [6]:
cat_dict

{'1': 'Film & Animation',
 '2': 'Autos & Vehicles',
 '10': 'Music',
 '15': 'Pets & Animals',
 '17': 'Sports',
 '18': 'Short Movies',
 '19': 'Travel & Events',
 '20': 'Gaming',
 '21': 'Videoblogging',
 '22': 'People & Blogs',
 '23': 'Comedy',
 '24': 'Entertainment',
 '25': 'News & Politics',
 '26': 'Howto & Style',
 '27': 'Education',
 '28': 'Science & Technology',
 '29': 'Nonprofits & Activism',
 '30': 'Movies',
 '31': 'Anime/Animation',
 '32': 'Action/Adventure',
 '33': 'Classics',
 '34': 'Comedy',
 '35': 'Documentary',
 '36': 'Drama',
 '37': 'Family',
 '38': 'Foreign',
 '39': 'Horror',
 '40': 'Sci-Fi/Fantasy',
 '41': 'Thriller',
 '42': 'Shorts',
 '43': 'Shows',
 '44': 'Trailers'}

In [7]:
youtube["category_id"].value_counts()

24    9964
10    6472
26    4146
23    3457
22    3210
25    2487
28    2401
1     2345
17    2174
27    1656
15     920
20     817
19     402
2      384
29      57
43      57
Name: category_id, dtype: int64

In [58]:
#only want popular videos:
youtube = youtube.loc[youtube["views"]>=200000]
#only get comedy/entertainment videos
youtube = youtube.loc[youtube["category_id"].isin([10, 22, 23, 24, 25])]

In [59]:
youtube.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [60]:
youtube.shape

(20181, 16)

In [61]:
youtube_title = youtube["title"].values.reshape(-1)

In [64]:
youtube_title

array(['WE WANT TO TALK ABOUT OUR MARRIAGE',
       'The Trump Presidency: Last Week Tonight with John Oliver (HBO)',
       'Racist Superman | Rudy Mancuso, King Bach & Lele Pons', ...,
       'BTS Plays With Puppies While Answering Fan Questions',
       'True Facts : Ant Mutualism',
       'I GAVE SAFIYA NYGAARD A PERFECT HAIR MAKEOVER BASED ON HER FEATURES: BTS! |bradmondo'],
      dtype=object)

In [13]:
import re
import string

youtube_title = [x.lower() for x in youtube_title]
youtube_title = [re.sub(r"\d+","", x) for x in youtube_title] #remove number
youtube_title = [re.sub(r"[^\w]", " ", x) for x in youtube_title] #remove symbol
shortword = re.compile(r"\W*\b\w{1,2}\b")
youtube_title = [shortword.sub("", x) for x in youtube_title] #remove words that <=2 characters

In [14]:
youtube_title

[' want talk about our marriage',
 'the trump presidency  last week tonight with john oliver  hbo ',
 'racist superman   rudy mancuso  king bach   lele pons',
 'nickelback lyrics  real fake ',
 ' dare you  going bald  ',
 'roy moore   jeff sessions cold open   snl',
 'why the rise the robots won mean the end work',
 'marshmello   blocks  official music video ',
 'which countries are about collapse ',
 'one change that would make pacific rim classic',
 'spaghetti burrito spaghetti burrito',
 'eminem   walk water  audio  beyoncé',
 'celebrities thanksgiving  ',
 'matthew santoro   facts  official music video  ellevan   humble the poet',
 'affairs boyfriends   million net worth   google   shawn and andrew',
 'elders react iphone  facial recognition  animojis ',
 'will watermarble   sister edition   watermarbling  random objects nail polish ',
 'jennifer lopez   amor  amor  amor  official video  wisin',
 'jason momoa wows hugh grant with some dothraki   the graham norton show',
 'daddy hom

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
youtube_title = [x+" seqend" for x in youtube_title]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(youtube_title)
seqs = tokenizer.texts_to_sequences(youtube_title)
word_index = tokenizer.word_index
seq_index = dict()
for key, val in word_index.items():
    seq_index[val] = key
#make input, output pair
input_target_pair = []
for s in seqs:
    for i in range(1, len(s)):
        input_target_pair.append(s[:i+1])

max_length = max([len(x) for x in seqs])

In [16]:
input_target_pair

[[400, 254],
 [400, 254, 24],
 [400, 254, 24, 79],
 [400, 254, 24, 79, 1578],
 [400, 254, 24, 79, 1578, 1],
 [2, 64],
 [2, 64, 3212],
 [2, 64, 3212, 66],
 [2, 64, 3212, 66, 108],
 [2, 64, 3212, 66, 108, 462],
 [2, 64, 3212, 66, 108, 462, 5],
 [2, 64, 3212, 66, 108, 462, 5, 51],
 [2, 64, 3212, 66, 108, 462, 5, 51, 1308],
 [2, 64, 3212, 66, 108, 462, 5, 51, 1308, 873],
 [2, 64, 3212, 66, 108, 462, 5, 51, 1308, 873, 1],
 [665, 974],
 [665, 974, 255],
 [665, 974, 255, 285],
 [665, 974, 255, 285, 121],
 [665, 974, 255, 285, 121, 752],
 [665, 974, 255, 285, 121, 752, 249],
 [665, 974, 255, 285, 121, 752, 249, 250],
 [665, 974, 255, 285, 121, 752, 249, 250, 1],
 [2099, 1309],
 [2099, 1309, 77],
 [2099, 1309, 77, 140],
 [2099, 1309, 77, 140, 1],
 [911, 9],
 [911, 9, 779],
 [911, 9, 779, 3545],
 [911, 9, 779, 3545, 1],
 [1023, 644],
 [1023, 644, 1138],
 [1023, 644, 1138, 3939],
 [1023, 644, 1138, 3939, 558],
 [1023, 644, 1138, 3939, 558, 336],
 [1023, 644, 1138, 3939, 558, 336, 44],
 [1023, 644

In [17]:
#split the pair into X and target
X = []
y = []
for s in input_target_pair:
    X.append(s[:-1])
    y.append(s[-1])
#pad the input
X = pad_sequences(X, maxlen=max_length, padding="post")
y = to_categorical(y)

In [18]:
X.shape

(132225, 18)

In [19]:
y.shape

(132225, 5503)

In [20]:
len(word_index)

5502

In [21]:
word_index["seqend"]

1

In [22]:
embedding = dict()
with open("./data/glove.6B.50d.txt", "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        embedding[word] = np.asarray(values[1:], dtype=np.float32)

In [23]:
embedding_weights = np.zeros((len(word_index) + 1, 50))
for word, i in word_index.items():
    embedding_vector = embedding.get(word)
    if embedding_vector is not None:
        embedding_weights[i] = embedding_vector

In [27]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, Dropout, Dense, GlobalMaxPool1D

K.clear_session()
model = Sequential()
model.add(
    Input(shape=(max_length,)))
model.add(
    Embedding(len(word_index)+1,
             50,
             weights=[embedding_weights],
             trainable=False))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(len(word_index)+1, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam")
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 18, 50)            275150    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               60400     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 5503)              555803    
Total params: 891,353
Trainable params: 616,203
Non-trainable params: 275,150
_________________________________________________________________


In [28]:
model.fit(X,y,epochs=5, batch_size=32)

Train on 132225 samples
Epoch 1/5
132225/132225 [==============================] - 20s 152us/sample - loss: 6.3749
Epoch 2/5
132225/132225 [==============================] - 18s 138us/sample - loss: 5.0147
Epoch 3/5
132225/132225 [==============================] - 18s 138us/sample - loss: 3.8902
Epoch 4/5
132225/132225 [==============================] - 18s 138us/sample - loss: 3.1406
Epoch 5/5
132225/132225 [==============================] - 18s 137us/sample - loss: 2.6581


In [33]:
def generate(seed_text, sentece_length):
    res = ""
    count = 0
    while (seed_text!="endseq") and (count<sentece_length):
        res += " " + seed_text
        seed_sequence = pad_sequences(tokenizer.texts_to_sequences([seed_text]), maxlen=max_length, padding="post")
        index = model.predict_classes(seed_sequence)
        seed_text = seq_index[index[0]]
        count+=1
    return res

In [66]:
print(generate("official", 5))
print(generate("ball", 3))
print(generate("united", 5))
print(generate("gordon", 6))
print(generate("shawn", 9))
print(generate("bts", 4))

 official video review the chainsmokers
 ball mix toast
 united korea act king masked
 gordon ramsay jenner monáe for couples
 shawn mendes bound old circle drag out your reflection
 bts 방탄소년단 official video
